In [1]:
import leafmap
from samgeo import SamGeo3, download_file
from samgeo.caption import ImageCaptioner

In [2]:
url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley.tif"
image_path = download_file(url)

uc_berkeley.tif already exists. Skip downloading. Set overwrite=True to overwrite.


In [3]:
m = leafmap.Map()
m.add_raster(image_path, layer_name="Satellite image")
m

Map(center=[37.87135, -122.25784999999999], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

In [4]:
captioner = ImageCaptioner(
    blip_model_name="Salesforce/blip-image-captioning-base",
    spacy_model_name="en_core_web_sm",
)

Loading weights:   0%|          | 0/472 [00:00<?, ?it/s]

In [5]:
caption, features = captioner.analyze(image_path)
print(f"Caption: {caption}")
print(f"Features: {features}")

Caption: an aerial view of a city with many buildings
Features: ['building']


In [6]:
from huggingface_hub import login
login()

In [9]:
sam3 = SamGeo3(backend="meta", device=None, checkpoint_path=None, load_from_HF=True)

ImportError: Meta SAM3 is not available. Please install it as:
	pip install segment-geospatial[samgeo3]

In [ ]:
sam3.set_image(image_path)

In [ ]:
feature = features[0]
sam3.generate_masks(prompt=feature)

In [ ]:
sam3.show_anns()

In [ ]:
sam3.show_masks()

In [ ]:
# Save masks with unique values for each object
# Since uc_berkeley.tif is a GeoTIFF, the output will also be a GeoTIFF
sam3.save_masks(output=f"{feature}_masks.tif", unique=True)

In [ ]:
# Save as binary mask (all foreground pixels are 255)
sam3.save_masks(output=f"{feature}_masks_binary.tif", unique=False)

In [ ]:
# Save masks and confidence scores
# Each pixel in the scores image will have the confidence value of its mask
sam3.save_masks(
    output=f"{feature}_masks_with_scores.tif",
    save_scores=f"{feature}_scores.tif",
    unique=True,
)

In [ ]:
sam3.show_masks(cmap="coolwarm")

In [ ]:
m.add_raster(f"{feature}_masks.tif", layer_name=f"{feature} masks", visible=False)
m.add_raster(
    f"{feature}_scores.tif",
    layer_name=f"{feature} scores",
    cmap="coolwarm",
    opacity=0.8,
    nodata=0,
)
m